In [13]:
#ingest data
import pandas as pd
import pickle
from sklearn.linear_model import Lasso



#For Evaluating the model
from sklearn.metrics import mean_absolute_error, mean_squared_error

#Algorithm used - LinearRegression
from sklearn.linear_model import LinearRegression

#To convert million , billion string into float
import re

#For plotting
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer

import seaborn as sns
import matplotlib.pyplot as plt

import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("youtube-prediction-experiment")

<Experiment: artifact_location='/workspaces/MLOps_Zoomcamp_Project_YoutubePrediction/02-Experiment_Tracking/mlruns/1', creation_time=1720106073958, experiment_id='1', last_update_time=1720106073958, lifecycle_stage='active', name='youtube-prediction-experiment', tags={}>

In [14]:
# Load the data
def read_dataframe(filename):
    data = pd.read_csv(filename)

    # Drop rows with NaN values
    data.dropna(inplace=True)
    
    data['VIEWS'] = clean_numeric_column(data['VIEWS'])
    data['TOTAL_NUMBER_OF_VIDEOS'] = clean_numeric_column(data['TOTAL_NUMBER_OF_VIDEOS'])
    data['SUBSCRIBERS'] = clean_numeric_column(data['SUBSCRIBERS'])
    
    print(data)
    
    
    return data


    
def clean_numeric_column(column):
    cleaned_column = []
    for value in column:
        if isinstance(value, str):
            if 'K' in value:
                cleaned_column.append(float(re.sub(r'[^0-9.]', '', value)) * 1000)
            elif 'M' in value:
                cleaned_column.append(float(re.sub(r'[^0-9.]', '', value)) * 1000000)
            elif 'B' in value:
                cleaned_column.append(float(re.sub(r'[^0-9.]', '', value)) * 1000000000)
            else:
                cleaned_column.append(float(re.sub(r'[^0-9.]', '', value)))
        else:
            cleaned_column.append(value)
    return cleaned_column

In [15]:
df_train = read_dataframe('/workspaces/MLOps_Zoomcamp_Project_YoutubePrediction/dataset/youtube_test.csv')
df_val = read_dataframe('/workspaces/MLOps_Zoomcamp_Project_YoutubePrediction/dataset/youtube_train.csv')

     RANK             NAME_OF_CHANNEL  TOTAL_NUMBER_OF_VIDEOS  SUBSCRIBERS  \
0      #1                     MrBeast                   799.0  274000000.0   
1      #2                    T-Series                 21120.0  267000000.0   
2      #3  Cocomelon - Nursery Rhymes                  1180.0  176000000.0   
3      #4                   SET India                138970.0  173000000.0   
4      #5           ✿ Kids Diana Show                  1220.0  123000000.0   
..    ...                         ...                     ...          ...   
491  #492       SUPER SLICK SLIME SAM                  1180.0   20600000.0   
492  #493              Clash of Clans                   868.0   20600000.0   
494  #495                 What's Idea                   205.0   20600000.0   
495  #496        Smile Family Spanish                   563.0   20500000.0   
496  #497        Marvel Entertainment                  8970.0   20500000.0   

            VIEWS                  CATEGORY  
0    5.098000e+10

In [16]:
print(len(df_train))
print(len(df_val))

485
485


In [17]:
# Define features and target
numeric_features = ['VIEWS', 'TOTAL_NUMBER_OF_VIDEOS']
categorical_features = ['CATEGORY']

# Prepare data for training
train_dicts = df_train[categorical_features + numeric_features].to_dict(orient='records')

# Prepare data for training
val_dicts = df_val[categorical_features + numeric_features].to_dict(orient='records')



# Initialize DictVectorizer
dv = DictVectorizer()


X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

print("X_train shape:", X_train.shape)

# Extract target variable
y_train = df_train['SUBSCRIBERS'].values
y_val = df_val['SUBSCRIBERS'].values
print("y_train shape:", y_train.shape)

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

X_train shape: (485, 34)
y_train shape: (485,)


/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


14801134.038355662

In [18]:
# Predict the number of subscribers for each channel in the dataset
df_val['Predicted Subscribers'] = lr.predict(X_val)

# Save the predictions to a new CSV file
df_val.to_csv('/workspaces/MLOps_Zoomcamp_Project_YoutubePrediction/resultset/Result_youtube_channels_with_predictions.csv', index=False)
print("Predictions saved to 'youtube_channels_with_predictions.csv'")

Predictions saved to 'youtube_channels_with_predictions.csv'


In [19]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Anamika")

    mlflow.log_param("train-data-path", "/workspaces/mlops_zoomcamp_project/dataset/youtube_train.csv")
    mlflow.log_param("valid-data-path", "/workspaces/mlops_zoomcamp_project/dataset/youtube_test.csv")
    

    alpha = 0.2
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="/workspaces/MLOps_Zoomcamp_Project_YoutubePrediction/02-Experiment_Tracking/models", artifact_path="models_pickle")

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.033255250603581e+16, tolerance: 26926150350515.47
  model = cd_fast.sparse_enet_coordinate_descent(
/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: '/workspaces/mlops_zoomcamp_project/models/lin_reg.bin'

In [ ]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [ ]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=10,
            evals=[(valid, 'validation')],
            early_stopping_rounds=2
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)